##Preprocessing 

Import necessary packages

In [0]:
pip install pycantonese

    100% |████████████████████████████████| 686kB 21.1MB/s 


In [0]:
import numpy as np
import pycantonese as pc
import pandas as pd
import jieba
import scipy
from pprint import pprint
import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1hBPhrxATm88P7glLNitOvPKdv7K7tNcz'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile("HKSocialMedia.csv")


    100% |████████████████████████████████| 993kB 13.6MB/s 


In [0]:
data = pd.read_csv("HKSocialMedia.csv")
posts = data['post_text']
print(posts)

0     據《經濟通》與《晴報》就《逃犯條例》舉行的網上民調，有83%支持修訂，而建議剔除《逃犯條例》...
1     高雄市長韓國瑜一邊「搶訂單、拼經濟」，一邊被民進黨扯後腿，民進黨仲羅織罪名誣衊佢「賣台」、「...
2     政府因應商界建議，昨天重新推出新修訂版的《逃犯條例》修改建議。民建聯立法會議員周浩鼎表示歡迎...
3     施梓山：「說來也好笑，主掌台灣陸委會的綠營中人見了涂謹申等人後，也真夠夜郎自大，香港修訂《逃...
4     何濼生：「特朗普對全球面對的挑戰如氣候變暖完全沒有興趣，卻想用強權的方法去尋求獲得最大的美國...
5     台籍全國政協委員凌友詩早前在全國政協會議上發言，力撐兩岸和平統一，卻遭台灣「內政部」罰款50...
6     隋文靜與韓聰日前在花滑世錦賽勇奪冠軍，為中國人爭光。而在較早前的表演滑中，兩人更以零失誤姿態...
7     肥佬黎又再賣資產救《壹傳媒》，4個月內第3度撲水，今次仲要出售「壹電視大樓」，相信《壹傳媒》...
8     甄子丹一家愛國情操令人感動！面對不公平的辱華行為，汪詩詩毫不貪戀名利場的享受，甘當領頭羊直接...
9     王國強：「香港雖然擁有優勢，但也不能以守株待兔的心態參與規劃，當前首要是丟棄各種被規劃論、被...
10    【林放之Blog】意大利總理不忘反擊批評者：英國、法國、德國早就爭著跟中國談生意，意大利算是...
11    陳日君上黃色港台節目、余若薇有份主持嘅《議事論事》，大讚年輕人佔中坐監有希望。咁都未算，佢仲...
12    反對派為抹黑中央竟散播白色恐怖，民主黨李永達亂噏，每日150名單程證中，有30至40人是「有...
13    【播音足本轉載】今集余非同大家分享對旺暴案梁天琦判決有4個不服氣，她指出香港好多案件的判決因...
14    【屈穎妍Blog】九個陪審員，五日車輪戰辯論，七比二的對決，即是說，有兩個人，一直力排眾議，...
15    「陳四萬」陳方安生一行人赴美面聖，引起大眾廣泛不滿。自由黨深水埗區議員李梓敬繼早前在Face...
16    陳建強：在中國人傳統觀念中是很嚴重的一回事，若連「死無對證」、「任由指罵」的往生者也被詛咒，...
17    【施永青Blog】歐洲的白人是憑甚麼有這麼突出的表現的呢？表面上可以看到的，是歐洲人

### Remove puncatuations

In [0]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '《', '》', '【', '】','▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '「','」','»', '！', '。','，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def remove_punctuation(x):
    x = str(x)
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, '')
    return x

texts = []
for line in posts:
    sentence = remove_punctuation(line)
    texts.append(sentence)


### Segmentation

In [0]:
# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1x_CcPBL8iowGWq7NpG7jrnEXoGKUdMD9'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('stopwords.txt')  

In [0]:
import codecs

def seg_word(sentence):
    #segmentation
    seg_list = jieba.cut(sentence)
    seg_result = []
    for w in seg_list:
        seg_result.append(w)
    #read stop_words
    stopwords = set()
    fr = codecs.open('stopwords.txt', 'r', 'utf-8')
    for word in fr:
        stopwords.add(word.strip())
    fr.close()
    return list(filter(lambda x: x not in stopwords, seg_result))

In [0]:
seg_res = []
for line in texts:
    new_line = seg_word(line)
    seg_res.append(new_line)

pprint(seg_res[:3])

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.121 seconds.
Prefix dict has been built succesfully.


[['經濟通',
  '晴報',
  '逃犯',
  '條例',
  '舉行',
  '網上民調',
  '83',
  '支持',
  '修訂',
  '建議',
  '剔除',
  '逃犯',
  '條例',
  '46',
  '項罪類',
  '中',
  '項有',
  '62',
  '表示',
  '不贊成',
  '網民',
  '精警',
  '留言',
  '召喚',
  '周日',
  '逃犯',
  '條例',
  '遊行',
  '黃',
  '之鋒',
  '看網',
  '上民調',
  '黃',
  '之鋒',
  '擘',
  '大眼',
  '睇',
  '清楚',
  '這網民',
  '講得',
  '好',
  '贊同',
  'Like'],
 ['高雄市',
  '長',
  '韓國瑜',
  '一邊',
  '搶訂',
  '單拼',
  '經濟',
  '一邊',
  '被民',
  '進黨',
  '扯後腿民進',
  '黨仲羅織',
  '罪名',
  '誣',
  '衊',
  '佢',
  '賣',
  '台',
  '犧',
  '牲',
  '台',
  '灣',
  '利益',
  '認真',
  '無恥',
  '眼光',
  '始終',
  '雪亮',
  '罪名',
  '只會',
  '暴露出',
  '民進',
  '黨心',
  '虛無能',
  '玻璃',
  '心碎',
  '一地',
  '一邊',
  '拼經濟',
  '搞好',
  '台灣',
  '一邊',
  '搞',
  '台獨',
  '走向',
  '死路',
  '精明',
  '台灣',
  '識得點',
  '揀',
  '搞',
  '台',
  '獨無運行',
  'Like'],
 ['政府',
  '因應',
  '商界',
  '建議',
  '昨天',
  '重新',
  '推出',
  '新修訂',
  '版',
  '逃犯',
  '條例',
  '修改',
  '建議',
  '民建',
  '聯',
  '立法',
  '會議員',
  '周浩鼎',
  '表示',
  '歡迎',
  '並指',
  '這是',
  '政府',
  '務實',
  '回應',
  '工商界',
  '要求',


## POS Tagging


### 1. Jieba

In [0]:
import jieba
import jieba.analyse
import jieba.posseg

In [0]:
# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1BNs01vAkSg5p7o2tLn-fEww6Bn6TGEk3'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('dict.txt.big.txt')

In [0]:
#set the traditional chinese dict as default
jieba.set_dictionary("dict.txt.big.txt")

In [0]:
def segment_all(sentence):
    sentence_seged = jieba.posseg.cut(sentence.strip())
    outstr = ''
    for x in sentence_seged:
        outstr+="{}/{},".format(x.word,x.flag)
    return outstr

In [0]:
text = "高雄市長韓國瑜一邊「搶訂單、拼經濟」，一邊被民進黨扯後腿，民進黨仲羅織罪名誣衊佢「賣台」、「犧牲台灣利益」，認真無恥！不過人民的眼光始終是雪亮的，這些「罪名」只會暴露出民進黨心虛無能，「玻璃心」碎了一地。一邊是拼經濟搞好台灣，一邊是搞台獨走向死路，精明的台灣人民識得點揀，搞台獨無運行Like！"
pprint(segment_all(text))


'高雄/nr,市長/x,韓國/x,瑜/nr,一邊/x,「/x,搶/x,訂單/x,、/x,拼/v,經濟/x,」/x,，/x,一邊/x,被/p,民進黨/x,扯後腿/x,，/x,民進黨/x,仲/zg,羅織罪名/x,誣衊/x,佢/x,「/x,賣台/v,」/x,、/x,「/x,犧牲/x,台灣/ns,利益/n,」/x,，/x,認真/x,無恥/x,！/x,不過/x,人民/n,的/uj,眼光/n,始終/x,是/v,雪亮/nr,的/uj,，/x,這些/x,「/x,罪名/n,」/x,只/d,會/v,暴露出/i,民進黨/x,心虛/x,無能/x,，/x,「/x,玻璃心/n,」/x,碎/a,了/ul,一地/s,。/x,一邊/x,是/v,拼/v,經濟/x,搞好/v,台灣/ns,，/x,一邊/x,是/v,搞/v,台/q,獨/v,走向/v,死路/n,，/x,精明/a,的/uj,台灣/ns,人民/n,識得/x,點/m,揀/v,，/x,搞/v,台/q,獨無/x,運行/x,Like/eng,！/x,'


In [0]:
jieba_postag = []
for line in seg:
  content = line[0]
  new_line = segment_all(content)
  jieba_postag.append(new_line)
  
pprint(jieba_postag)

['經濟/x,通/v,',
 '高雄市/ns,',
 '政府/n,',
 '施梓/nrfg,山/n,',
 '何濼生/nr,',
 '台籍/n,',
 '隋/nr,文靜/x,',
 '肥佬黎/ns,',
 '甄子丹/nr,',
 '王國強/x,',
 '林放/nr,',
 '陳/x,',
 '反對/x,',
 '播音/n,',
 '屈穎妍/nr,',
 '陳/x,',
 '陳/x,',
 '施永青/nr,',
 '美軍/x,打/v,',
 '屈穎妍/nr,',
 '特首/n,',
 '美國/x,',
 '英國/x,',
 '博鰲/x,亞洲/x,',
 '特區/x,',
 '正在/t,',
 '歐盟/x,',
 '本土/n,',
 '高雄市/ns,',
 '越南政府/nt,',
 '美國/x,',
 '歐盟/x,',
 '金澤培/nr,強/v,',
 '行動/x,',
 '唔/y,',
 '原來/x,',
 '死者/n,',
 '新系/n,',
 '梁/nr,',
 '劉兆佳/nr,',
 '焦點/x,',
 '焦點/x,涉/v,',
 '焦點/x,',
 '點播/x,',
 '點播/x,',
 '焦點/x,',
 '焦點/x,',
 '焦點/x,',
 '點播/x,',
 '點/zg,',
 '外貌/n,',
 '屈穎妍/nr,',
 '特金/n,會/v,',
 '機械/x,',
 '蜘蛛/n,',
 '港鐵/n,',
 '獨/zg,人/n,',
 '陳/x,',
 '大圍/n,',
 '阿嬌/nr,',
 '罷工/x,',
 '想/v,',
 '美國/x,',
 '國歌/x,',
 '香港/ns,',
 '港長/n,',
 '港/j,獨/v,',
 '無須/x,',
 '香港/ns,',
 '晨讀/x,',
 '英國/x,脫/x,',
 '影響/x,',
 '中國/x,',
 '習近/v,',
 '英國/x,',
 '英聞/nt,',
 '習近平/x,',
 '習近平/x,',
 '英國/x,冀脫/ns,',
 '影響/x,',
 '表證/v,',
 '點/m,睇/vg,',
 '韓國/x,瑜/nr,',
 '陳/x,',
 '高雄市/ns,',
 '人命/n,',
 '可喜/v,',
 '法律/n,',
 '務實/x,',
 '特區/x,',
 '保衛/x,',

### 2. THULAC

In [0]:
pip install thulac

    100% |████████████████████████████████| 52.9MB 675kB/s 
  Stored in directory: /root/.cache/pip/wheels/d2/3c/d4/2af73c09df43b30a03c280c85c5858b2744edf5d62edb5457b
Successfully built thulac


In [0]:
import thulac

thu_model = thulac.thulac()
thu_postag = []
for line in texts:
    new_line = thu_model.cut(line)
    thu_postag.append(new_line)


Model loaded succeed


n/名词 np/人名 ns/地名 ni/机构名 nz/其它专名
m/数词 q/量词 mq/数量词 t/时间词 f/方位词 s/处所词
v/动词 vm/能愿动词 vd/趋向动词 a/形容词 d/副词
h/前接成分 k/后接成分 i/习语 j/简称
r/代词 c/连词 p/介词 u/助词 y/语气助词
e/叹词 o/拟声词 g/语素 w/标点 x/其它 

In [0]:
pprint(thu_postag)

[[['據經濟', 'n'],
  ['通與', 'v'],
  ['晴報', 'n'],
  ['就', 'd'],
  ['逃犯', 'v'],
  ['條例', 'n'],
  ['舉行', 'v'],
  ['的', 'u'],
  ['網', 'v'],
  ['上', 'v'],
  ['民調', 'n'],
  ['有', 'v'],
  ['83', 'm'],
  ['支持', 'v'],
  ['修訂', 'v'],
  ['而', 'c'],
  ['建議', 'v'],
  ['剔除', 'v'],
  ['逃犯', 'n'],
  ['條例', 'n'],
  ['46', 'm'],
  ['項', 'q'],
  ['罪類', 'n'],
  ['中', 'f'],
  ['的', 'u'],
  ['9', 'm'],
  ['項', 'q'],
  ['有', 'v'],
  ['62', 'm'],
  ['表示', 'v'],
  ['不', 'd'],
  ['贊', 'v'],
  ['成', 'v'],
  ['對', 'g'],
  ['此', 'r'],
  ['有', 'v'],
  ['網民', 'n'],
  ['就', 'd'],
  ['精警', 'a'],
  ['地', 'u'],
  ['留言', 'v'],
  ['召喚', 'v'],
  ['周日', 't'],
  ['就', 'p'],
  ['逃犯', 'n'],
  ['條例', 'n'],
  ['遊行', 'v'],
  ['的', 'u'],
  ['黃', 'g'],
  ['之', 'u'],
  ['鋒', 'g'],
  ['叫', 'v'],
  ['他', 'r'],
  ['看網', 'v'],
  ['上', 'f'],
  ['民調', 'n'],
  ['結果黃', 'n'],
  ['之', 'u'],
  ['鋒', 'g'],
  ['擘', 'v'],
  ['大', 'a'],
  ['眼', 'n'],
  ['睇', 'v'],
  ['清楚', 'a'],
  ['啦', 'u'],
  ['這網民', 'np'],
  ['講', 'v'],
  ['得', 'u'],
  ['好', 'a'],

In [0]:
# do pos-tagger on segmented results
seg_pos_tag = []
for line in seg_res:
    content = line[0]
    new_line = thu_model.cut(content)
    seg_pos_tag.append(new_line)
    
pprint(seg_pos_tag)

[[['經濟', 'n']],
 [['高雄', 'ns']],
 [['政府', 'n']],
 [['施梓山', 'np']],
 [['何濼生', 'np']],
 [['台籍', 'n']],
 [['隋', 'g']],
 [['肥佬黎', 'ns']],
 [['甄子丹', 'np']],
 [['王國強', 'np']],
 [['林放', 'np']],
 [['陳日君', 'np']],
 [['反對派', 'n']],
 [['播音', 'v']],
 [['屈穎妍', 'np']],
 [['陳', 'p']],
 [['陳', 'p']],
 [['施永青', 'np']],
 [['美軍', 'n']],
 [['屈穎妍', 'np']],
 [['特首', 'j']],
 [['美國', 'n']],
 [['英國', 'q'], ['外交部', 'ni']],
 [['博鰲', 'j']],
 [['特區', 'a'], ['政府', 'n']],
 [['正在', 'd']],
 [['歐盟', 'j']],
 [['本土', 'n']],
 [['高雄', 'ns']],
 [['越南', 'ns'], ['政府', 'n']],
 [['美國', 'n']],
 [['歐盟', 'j']],
 [['金澤培', 'np']],
 [['行動', 'n']],
 [['唔', 'u']],
 [['原來', 'n']],
 [['死者', 'n'], ['家', 'n'], ['屬', 'v']],
 [['新', 'a']],
 [['梁', 'np']],
 [['劉兆佳', 'np']],
 [['焦點', 'a']],
 [['焦點', 'a']],
 [['焦點', 'a']],
 [['點播', 'v']],
 [['點播', 'v']],
 [['焦點', 'a']],
 [['焦點', 'a']],
 [['焦點', 'a']],
 [['點播', 'v']],
 [['點', 'p']],
 [['外貌', 'n']],
 [['屈穎妍', 'np']],
 [['特金會', 'n']],
 [['機械', 'n']],
 [['蜘蛛俠', 'n']],
 [['港鐵', 'n']],
 [['人', 'n']],

##Build Cantonese sentiment lexicon

The cantonoese sentiment lexicon is built based on a Cantonese-English Dictionary. Three different models are used to calculate the sentiment score.

In [0]:
# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1Wy75DNIEOx8rpYBVQVU-TP4nDTnlC_16'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('cccanto-webdist.txt')  

In [0]:
# read the original canto-eng dict in
with open("cccanto-webdist.txt", encoding="utf8") as f:
    content = [line.split("/") for line in f]

#remove irrelenvent information   
words_list = content[13:]

###1. NLTK sentiment analysis package

In [0]:
def nltk_sen_score(lst):
    sid = SentimentIntensityAnalyzer()
    sentence_list = []
    for sentence in lst:
        sentence_list = ','.join(lst)
    sen_score = sid.polarity_scores(sentence_list)
    return sen_score
    

The nltk package provides Vader for sentiment analysis, and the sentiment score shows in four different dimensions. The following is an example of a result generated by Vader.

In [0]:
nltk.download('vader_lexicon')
example = ["to die and end it all, when one chooses to take his/ her life, death pays all debts"]
example_score = nltk_sen_score(example)
print(example_score)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
{'neg': 0.328, 'neu': 0.672, 'pos': 0.0, 'compound': -0.8316}


In [0]:
def remove_romanization(line):
    words = line.split()
    first_word = words[0]
    return first_word

In [0]:
nltk_sen_dict ={}
for lst in words_list:
    definition_list =[]
    try:
        for idx, elem in enumerate(lst):
            if len(elem) > 2:
                if idx != 0:
                    definition_list.append(elem)
                else:
                    word_meaning = remove_romanization(elem)
                    nltk_sen_dict[word_meaning] = None
        sscore = nltk_sen_score(definition_list)
        full_word = lst[0]
        word_key = remove_romanization(full_word)
        nltk_sen_dict[word_key] = sscore
    except:
        pass

In [0]:
dict(list(nltk_sen_dict.items())[0:10])

{'一不小心': {'compound': -0.34, 'neg': 1.0, 'neu': 0.0, 'pos': 0.0},
 '一世人': {'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0},
 '一世英名': {'compound': 0.7906, 'neg': 0.0, 'neu': 0.3, 'pos': 0.7},
 '一个': {'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0},
 '一串': {'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0},
 '一九九七年': {'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0},
 '一九四九年': {'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0},
 '一事': {'compound': 0.0258, 'neg': 0.0, 'neu': 0.0, 'pos': 1.0},
 '一二': {'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0},
 '一些些': {'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0}}

###2. TextBlob sentiment analysis package

In [0]:
from textblob import TextBlob

def tb_sen_score(lst):
    sentence_list = []
    for sentence in lst:
        sentence_list = ','.join(lst)
    definition = TextBlob(sentence_list)
    sen_score = definition.sentiment
    return sen_score

The sentiment property returns a namedtuple of the form Sentiment(polarity, subjectivity). The polarity score is a float within the range [-1.0, 1.0]. The subjectivity is a float within the range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective.

In [0]:
opinion = TextBlob( "to die and end it all, when one chooses to take his/ her life, death pays all debts")
score = opinion.sentiment
print(score)

Sentiment(polarity=0.0, subjectivity=0.0)


In [0]:
tb_sen_dict = {}
for lst in words_list:
    definition_list =[]
    try:
        for idx, elem in enumerate(lst):
            if len(elem) > 2:
                if idx != 0:
                    definition_list.append(elem)
                else:
                    tb_sen_dict[elem] = None
        sscore = str(tb_sen_score(definition_list))
        word_key = lst[0]
        tb_sen_dict[word_key] = sscore
    except:
        pass

In [0]:
dict(list(tb_sen_dict.items())[0:10])

{'一不小心 一不小心 [yi1 bu5 xiao3 xin1] {jat1 bat1 siu2 sam1} ': 'Sentiment(polarity=0.0, subjectivity=0.0)',
 '一世人 一世人 [yi1 shi4 ren2] {jat1 sai3 jan4} ': 'Sentiment(polarity=0.1, subjectivity=0.5125)',
 '一世英名 一世英名 [yi1 shi4 ying1 ming2] {jat1 sai3 jing1 ming4} ': 'Sentiment(polarity=0.8, subjectivity=0.75)',
 '一个 一个 [yi1 ge5] {jat1 go3} ': 'Sentiment(polarity=0.0, subjectivity=0.0)',
 '一串 一串 [yi1 chuan4] {jat1 cyun3} ': 'Sentiment(polarity=0.0, subjectivity=0.0)',
 '一九九七年 一九九七年 [yi1 jiu3 jiu3 qi1 nian2] {jat1 gau2 gau2 cat1 nin4} ': 'Sentiment(polarity=0.0, subjectivity=0.0)',
 '一九四九年 一九四九年 [yi1 jiu3 si4 jiu3 nian2] {jat1 gau2 sei3 gau2 nin4} ': 'Sentiment(polarity=0.0, subjectivity=0.0)',
 '一事 一事 [yi1 shi4] {jat1 si6} ': 'Sentiment(polarity=0.0, subjectivity=0.0)',
 '一二 一二 [yi1 er4] {jat1 ji6} ': 'Sentiment(polarity=-0.19375, subjectivity=0.3)',
 '一些些 一些些 [yi1 xie1 xie1] {jat1 se1 se1} ': 'Sentiment(polarity=-0.1875, subjectivity=0.5)'}

## Cantonese-Mandarin dict based sentiment lexicon


###1. Bosen NLP

In [0]:
# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1TPnNJjwjxa4hAtAeJJN6o0oFc7eJ_jGm'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('BosonNLP_sentiment_score.txt')  

In [0]:
with open("BosonNLP_sentiment_score.txt", encoding="utf8") as f:
    content = f.readlines()
 

In [0]:
example = content[:30]
pprint(example)

['最尼玛 -6.70400012637\n',
 '扰民 -6.49756445867\n',
 'fuck... -6.32963390433\n',
 'RNM -6.21861284426\n',
 'wcnmlgb -5.96710044003\n',
 '2.5: -5.90459648251\n',
 'Fxxk -5.87247473641\n',
 'MLP -5.87247473641\n',
 '吃哑巴亏 -5.77120419579\n',
 'IAQI -5.77107837123\n',
 'MLGBD -5.69408191501\n',
 'NNND -5.66228462641\n',
 'MLGB. -5.60457743583\n',
 '成甘 -5.60457743583\n',
 '最桑 -5.60457743583\n',
 '真无语 -5.60457743583\n',
 'ＴＭ -5.60457743583\n',
 '次奥次奥次奥 -5.59258287133\n',
 'cnmd -5.54446545761\n',
 'MBD -5.50280109843\n',
 'NNDX -5.48173951768\n',
 '水蛭 -5.48173951768\n',
 '美素丽 -5.48173951768\n',
 '草尼 -5.48173951768\n',
 '凌迟 -5.46005372985\n',
 '尼玛尼玛尼玛 -5.42622557462\n',
 '冠周炎 -5.41616190446\n',
 '加塞儿 -5.41616190446\n',
 '日尼玛 -5.41616190446\n',
 '朝仲 -5.41616190446\n']


## Chinese sentiment lexicon
1. Bosen NLP sentiment lexicon (https://bosonnlp.com/dev/resource)
Including 110k phrases, also some unformally words.
2. Baidu api (https://cloud.baidu.com/doc/NLP/NLP-API/30.5C.69.65.EA.9A.5B.DB.98.A4.9F.5D.DF.1F.B8.CD.AC.05.html#.E8.AF.B7.E6.B1.82.E8.AF.B4.E6.98.8E )
5qps/day
3. Open source sentiment bridge (https://github.com/rainarch/SentiBridge)
4. NTUSD sentiment lexicon (traditional Chinese) (http://academiasinicanlplab.github.io/)
5. Tencent NLP api (https://nlp.qq.com/)
6. Sentiment Ontology (http://ir.dlut.edu.cn/EmotionOntologyDownload)
7. AC NLP (http://www.acnlp.com/acnlp-demo)

## Sentiment Analysis
Using the nltk sentiment lexicon we just built to perform an sentiment analysis on the facebook posts.  For each post, we match all the words in the sentiment lexicon with a score, then use average score of all the words as the final score for that post.

In [0]:
def get_sen_score(lst):
    score_list = []
    avg_dict = {}
    for word in lst:
        if word in nltk_sen_dict:
            score = nltk_sen_dict[word]
            score_list.append(score)
    for score_dict in score_list:
        for key in score_dict:
             if key not in avg_dict:
                 avg_dict[key] = score_dict[key]
             else:
                 avg_dict[key] = avg_dict[key] + score_dict[key]
    for key in avg_dict:
        avg_dict[key] = avg_dict[key]/len(score_list)
    return avg_dict

In [0]:
posts_sen_score = {}       
for idx, post in enumerate(seg_res):
    score = get_sen_score(post)
    full_post = posts[idx]
    posts_sen_score[full_post] = score

In [0]:

dict(list(posts_sen_score.items())[0:20])

{'「陳四萬」陳方安生一行人赴美面聖，引起大眾廣泛不滿。自由黨深水埗區議員李梓敬繼早前在Facebook怒轟他們「嘔心（噁心）」後，接受建制媒體電話訪問繼續鬧爆，直斥「好睇唔過眼」。李梓敬事後義憤未息，在Facebook指：「都係嗰句，最睇唔過眼嘅，就係嗰啲專幫外國人，打返自己人嘅政客！」李梓敬三番四次批評陳四萬一行，講出我心聲，更可見其不平則鳴的本色。香港正需要這些為理發聲的議員！': {'compound': -0.005689473684210543,
  'neg': 0.062000000000000006,
  'neu': 0.893684210526316,
  'pos': 0.04431578947368422},
 '【屈穎妍Blog】九個陪審員，五日車輪戰辯論，七比二的對決，即是說，有兩個人，一直力排眾議，堅定不移地擁抱真理。他們的身份不會被公佈，沒人知道這兩位以二敵七的勇者是誰，但他們為法律守住最後一道防線，雖然最後敗陣，絕對問心無愧。我們常說社會上太多沉默大多數，有時想想，他們沉默，會否因為發了正義之聲、做過正確之事，卻得不到大家的迴響和認同，於是，下次就選擇沉默了？正如這兩位無名無姓的陪審員，他們堅持到最後，又可有得到大眾致敬的掌聲？謹在小欄，衷心向不認識的你倆致謝！': {'compound': 0.011019999999999997,
  'neg': 0.0796,
  'neu': 0.8892,
  'pos': 0.0312},
 '【屈穎妍blog】試想想，如果香港建制派議員上京會見國安局官員，你道反對派會反彈到甚麼程度？一個高雄市長帶著一大棚人進中聯辦坐坐都是「賣台」，陳方安生、郭榮鏗、莫乃光三人前倨後恭握著美國國家安全部官員的手，賣的又是甚麼？': {'compound': 0.146475,
  'neg': 0.0,
  'neu': 0.92575,
  'pos': 0.07425},
 '【播音足本轉載】今集余非同大家分享對旺暴案梁天琦判決有4個不服氣，她指出香港好多案件的判決因官而異，標準不統一的情況嚴重，讓人感到不安。余非認為應進行司法改革設立監督機制，不要讓改革力量變成別有用心者的政治子彈。': {'compound': -0.01913999999999999,
  'neg': 0.146,
  

# NER

In [0]:
pip install wikipedia

In [0]:
# we consider noun and person's name as possible named entities 
named_entity_list = []
for lst in thu_postag:
    for line in lst:
        tag = line[-1]
        print(tag)
        word =line[0]
        print(word)
        #single words hardly become named entities, so we remove it
        if len(word) > 1:
            if tag == 'n':
                named_entity_list.append(word)
            elif tag == 'np':
                named_entity_list.append(word)
            else: 
                pass
        
#remove repeated words in the list       
unique_ent_list = list(set(named_entity_list))

In [0]:
#try to sereach the word in wikipedia and gets its categories& summary
#if its page exists     
import wikipedia
wikipedia.set_lang("zh-yue")
def get_wiki_category(word):
    content = wikipedia.page(title=word, pageid=None, auto_suggest=False, 
               redirect=False, preload=False)
    category = content.categories
    return category


def get_wiki_des(word):
    description = wikipedia.summary(word, sentences=1)
    return description


In [0]:
#get the description and categories of a word and store them in a dict
entities = {}
for word in a:
    try: 
        cg = get_wiki_category(word)
        des = get_wiki_des(word)
        if word not in entities:
            entities[word] = {}
            entities[word]["category"] = cg
            entities[word]["description"] = des           
    except:
        pass

In [0]:
#Insert data into database
import pymssql
conn = pymssql.connect(
    server = "cantonesenlp.cavc351ulyly.ap-southeast-2.rds.amazonaws.com", 
    user = "jac_research", 
    password = "NLPresearch", 
    database = "CantoneseNLP")

try:
    with connection.cursor() as cursor:
        # Create a new record
        sql = "INSERT INTO `des` (`ID`) VALUES (%s, %s)"
        cursor.execute(sql, ('webmaster@python.org', 'very-secret'))

    # connection is not autocommit by default. So you must commit to save
    # your changes.
    connection.commit()

finally:
    connection.close()